In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

db = pd.read_csv('../datasets/top10stock.csv')

select = db.iloc[:, 0] == "AAPL"
apple = db.loc[select, :]

In [2]:
# Reformatting data, datapreprocessing
for row in range(2516):
    for col in range(8):
        if col in [2, 4, 5, 6]:
            apple.iloc[row,col] = apple.iloc[row,col].strip("$")

In [3]:
date_objects = []
for strp in apple.Date:
    obj = None
    if strp[2] == "/":
        obj = datetime.strptime(strp, "%m/%d/%Y")
    else:
        obj = datetime.strptime(strp, "%m-%d-%Y")
    date_objects.append(obj)
days = pd.DataFrame([date_objects[i].day for i in range(len(date_objects))])
months = pd.DataFrame([date_objects[i].month for i in range(len(date_objects))])
years = pd.DataFrame([date_objects[i].year for i in range(len(date_objects))])
days.columns = ["Day"]
months.columns = ["Month"]
years.columns = ["Year"]

# splitting variables into its features and labels
features = pd.concat([days, months, years, apple.iloc[:, [3,4,5,6]]], axis=1)
labels = pd.DataFrame(apple.iloc[:,2])

# Converting their datatypes
features = features.astype('float32')
labels = labels.astype('float32')

In [4]:

# hyperparameters
epochs = 4
batchsize = 2
lr = 0.0001
inputsize = 7
hid_layer1_sz = 16
hid_layer2_sz = 16
hid_layer3_sz = 16
class_size = 1

In [5]:
# Scaling our data
ftmean = np.nanmean(features)
ftstd = features.values.std()
lbmean = np.nanmean(labels)
lbstd = labels.std()

features = pd.DataFrame((features.values - ftmean) / ftstd)
labels = (labels - lbmean) / lbstd

In [6]:
# Creating a database class that the dataloader can use. (Dataloader is the what we will be using to train and test)
class database(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features.iloc[idx].values
        label = self.labels.iloc[idx].values
        feature = torch.tensor(feature)
        label = torch.tensor(label)
        return feature, label



In [7]:
# Splitting the data into training data and testing data
xtrain, xtest, ytrain, ytest = train_test_split(features,labels,train_size=0.8)

# initializing our database class with the splitted data
train_db = database(xtrain,ytrain)
test_db = database(xtest,ytest)

# Creating our dataloader and transforming it into batches, etc.
trainloader = torch.utils.data.DataLoader(train_db, batch_size=batchsize, shuffle=False)
testloader = torch.utils.data.DataLoader(test_db, batch_size=batchsize, shuffle=False)

In [8]:
2012/503

4.0

In [9]:

# Creating our model architechture
class stock_classification(nn.Module):
    def __init__(self, inputsize, hidden1, hidden2, hidden3, class_sz):
        super(stock_classification, self).__init__()
        self.l1 = nn.Linear(inputsize,hidden1)
        self.activation = nn.ReLU()
        self.l2 = nn.Linear(hidden1, hidden2)
        self.l3 = nn.Linear(hidden2, hidden3)
        self.l4 = nn.Linear(hidden3, class_sz)

    def forward(self, x):
        out = self.l1(x)
        out = self.activation(out)
        out = self.l2(out)
        out = self.activation(out)
        out = self.l3(out)
        out = self.activation(out)
        out = self.l4(out)
        return out

In [10]:
# Initializing our model, loss function, and optimizer
model = stock_classification(inputsize, hid_layer1_sz,hid_layer2_sz,hid_layer3_sz,class_size).to(device)
model.train()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [11]:
# Training our model:

n_total_steps = len(trainloader)
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 4 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')

Epoch [1/4], Step [4/1006], loss: 1.8806
Epoch [1/4], Step [8/1006], loss: 0.7162
Epoch [1/4], Step [12/1006], loss: 0.5354
Epoch [1/4], Step [16/1006], loss: 0.1054
Epoch [1/4], Step [20/1006], loss: 0.3456
Epoch [1/4], Step [24/1006], loss: 3.2668
Epoch [1/4], Step [28/1006], loss: 0.2845
Epoch [1/4], Step [32/1006], loss: 0.5348
Epoch [1/4], Step [36/1006], loss: 0.2199
Epoch [1/4], Step [40/1006], loss: 1.9628
Epoch [1/4], Step [44/1006], loss: 1.2648
Epoch [1/4], Step [48/1006], loss: 1.9084
Epoch [1/4], Step [52/1006], loss: 1.6385
Epoch [1/4], Step [56/1006], loss: 1.9183
Epoch [1/4], Step [60/1006], loss: 3.1255
Epoch [1/4], Step [64/1006], loss: 1.3268
Epoch [1/4], Step [68/1006], loss: 1.8466
Epoch [1/4], Step [72/1006], loss: 0.2329
Epoch [1/4], Step [76/1006], loss: 0.3404
Epoch [1/4], Step [80/1006], loss: 0.3378
Epoch [1/4], Step [84/1006], loss: 1.2260
Epoch [1/4], Step [88/1006], loss: 0.3966
Epoch [1/4], Step [92/1006], loss: 1.3459
Epoch [1/4], Step [96/1006], loss: 0

In [12]:
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = len(testloader.dataset)
    for inputs, labels in testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        outputs = model(inputs)
        a, predicted = torch.max(outputs, 1)
        n_correct += (abs(predicted - labels) < 0.02).sum().item()
    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test closing prices: {100*acc:.4f} %')

for inputs, labels in testloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    print(f"outputs: {outputs}\n labels: {labels}")
    break



Accuracy of the network on the 504 test closing prices: 1.1905 %
outputs: tensor([[-0.6427],
        [-0.5553]], device='cuda:0', grad_fn=<AddmmBackward0>)
 labels: tensor([[-0.8844],
        [-0.4102]], device='cuda:0')
